In [9]:
pip install gensim

     |████████████████████████████████| 24.7 MB 22.9 MB/s eta 0:00:01   |████                            | 3.0 MB 2.1 MB/s eta 0:00:11
     |████████████████████████████████| 99 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 19.7 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 12.9 MB/s eta 0:00:01     |████████████▋                   | 2.4 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 7.2 MB/s  eta 0:00:01
     |████████████████████████████████| 87 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 9.2 MB/s  eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.10.0-py3-none-any.whl size=90632 sha256=3f317eb6b8a155d6cb58

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import scipy
import sklearn
import spacy
import re

import nltk
from nltk.corpus import gutenberg, stopwords
nltk.download('gutenberg')  
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/vilandao/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text[0:900000]


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

In [4]:
# Parse the data.
nlp = spacy.load('en')
austen_doc = nlp(austen_clean)

In [7]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in austen_doc.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)

print(sentences[15])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(austen_clean)))

['intimate', 'friend', 'sensible', 'deserve', 'woman', 'bring', 'strong', 'attachment', 'settle', 'close', 'village', 'kellynch', 'kindness', 'advice', 'lady', 'elliot', 'mainly', 'rely', 'good', 'help', 'maintenance', 'good', 'principle', 'instruction', 'anxiously', 'give', 'daughter']
We have 8323 sentences and 900000 tokens.


In [11]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=2,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=15,  # Minimum word count threshold.
    window=10,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=500,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [13]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
print('')

# Similarity is calculated using the cosine, so again 1 is total similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))
print('')

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('room', 0.9864879846572876), ('cousin', 0.9795949459075928), ('bates', 0.978899359703064), ('talk', 0.9783236980438232), ('confidence', 0.9756678342819214), ('burst', 0.9752973914146423), ('join', 0.9726870059967041), ('sister', 0.9726129770278931), ('meet', 0.9720394611358643), ('speak', 0.9712566137313843)]

0.7104795

marriage


/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if sys.path[0] == '':


In [15]:
# Load Google's pre-trained Word2Vec model.
model_google = gensim.models.KeyedVectors.load_word2vec_format ('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# List of words in model.
vocab = model_google.wv.vocab.keys()

print(model_google.wv.most_similar(positive=['man', 'king'], negative=['woman']))
print('')




/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/Users/vilandao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
